### Boosting
- 给定一组训练样本：$(x_1, y_1), (x_2, y_2), \cdots ,(x_n, y_n)$, 目标是期望找到近似函数$\hat f(\vec x)$, 使得损失函数 $L(y,\hat f_m(\vec x)$ 尽可能小。
- 假定为$\hat f(\vec x)$ 为一族函数的加权和，$$\hat f(\vec x) = \sum_{i=1}^M \gamma_i h_i(\vec x) + const $$
- 可以认为，这是以贪心法的思路扩展得到了一族函数 $$f_m(\vec x) = f_{m-1}(\vec x) + argmin_{h\in H}L(y,f_{m−1}(\vec x)+h_m(\vec x))$$
- 不同损失函数及其最优解如下：
<img src="image/boost_category.png" width = "800" height = "600" alt="GBDT" align=center />
<img src="image/loss_function.png" width = "600" height = "400" alt="GBDT" align=center />
 - 损失函数为绝对值误差、平方误差则为回归问题
 - Logloss、指数损失则可以处理分类问题
 - Huber损失和分位数损失，主要用于健壮回归，也就是减少异常点对损失函数的影响。
- 容易证明：绝对值最小化的最优解为中位数
 - 不失一般性，假设$x_i$ 升序排列，
$$J(u) = \sum_{i=1}^n|x_i - u| = \sum_{i=1}^k(u-x_i) +  \sum_{i=k+1}^n(x_i - u)$$
 - 求偏导，令导数为0.
 $$\frac{\partial J(u)}{\partial u} = \sum_{i=1}^k (1) +  \sum_{i=k+1}^n (-1) \rightarrow 0$$
 - 从而可以看到前$k$ 个数和后$n -k$ 个数数目相等，即$u$ 为中位数

#### GBDT
- GBDT也是集成学习Boosting家族的成员，但是却和传统的Adaboost有很大的不同。Adaboost 是利用前一轮迭代弱学习器的误差率来更新训练集的权重，这样一轮轮的迭代下去。GBDT也是迭代，使用了前向分布算法，但是弱学习器限定了**只能使用CART回归树模型**，同时迭代思路和Adaboost也有所不同。
- 在GBDT的迭代中，假设前一轮迭代得到的强学习器是$f_{m−1}(\vec x)$, 可以认为预测值即位$\hat y =f_{m−1}(\vec x)$，损失函数是$L(y, f_{m−1}(\vec x))$, 本轮迭代的目标是找到一个CART回归树模型的弱学习器$h_m(\vec x)$，让本轮的损失损失$L(y,f_m(\vec x)=L(y,f_{m−1}(\vec x)+h_m(\vec x))$最小。也就是说，本轮迭代找到决策树，要让样本的损失尽量变得更小。
- 所谓的Gradient Boosting 其实就是在更新的时候选择**负梯度方向**来保证最后的结果最好
* GBDT 算法描述如下：

<img src="image/GBDT.png" width = "600" height = "400" alt="GBDT" align=center />

* 可以看到，初始模型给定为常数 $f_0(\vec x)$, 之后做$M$ 次迭代，生成$M$ 棵树回归树。
* a 步第$m$ (t)轮的第$i$个样本的损失函数的负梯度表示
* b 步主要使用数据${(x_i, r_{im})}$ 计算**拟合残差**的基函数$f_m(\vec x)$
* c 步主要计算最优步长$\gamma_{jm}$, 可以使用一维搜索的方式优化, 本轮最优的决策树函数为： $h_m(x) =\sum_{j=1}^{J_m}\gamma_{jm}I(x\in R_{jm})$
* d 步主要更新模型： $f_m(x) = f_{m-1}(x) +  \sum_{j=1}^{J_m}\gamma_{jm}I(x\in R_{jm})$

#### xgboost
* 在GBDT 的基础上考虑二阶导数
* 目标函数： $J(f_t) = \sum_{i=0}^n L(y_i, \hat y_i^{(t-1)}+f_t(x_i)) + \Omega(f_t) + C$
* 根据Taylor 二阶展式：
   $$ f(x+ \Delta x) \approx f(x) + f(x)'\Delta x + {1\over2} f(x)''\Delta x^2 $$
   * 令 $$g_i = \frac{\partial L\left(y_i, \hat y_i^{\left(t-1\right)}\right)}{\partial  \hat y_i^{\left(t-1\right)} }$$, 
   $$h_i = \frac{\partial^2 L\left(y_i, \hat y_i^{\left(t-1\right)}\right)}{\partial  \hat y_i^{\left(t-1\right)} }$$
   则目标函数可以近似如下：
   $$J(f_t) \approx \sum_{i=0}^n L(y_i, \hat y_i^{(t-1)}) + g_if_t(x_i) + {1\over2}h_if_t^2(x_i) + \Omega(f_t) + C$$
* 决策树核心： 树结构 $q(x)$: 样本落在某个叶子节点， 叶权重： $w$， $T$ 为叶子个数: 
$$f_t(x) = w_{q(x)}, w\in R^T, q: R^d\rightarrow \in \{1,2,\cdots , T\}$$

* 复杂度：
$$\Omega(f_t)  = \gamma T + {1\over 2}\lambda \sum_{j=1}^T w_j^2  $$

* 目标函数重写如下：
  $$\begin{align}
  J(f_t) & \approx \sum_{i=0}^n \left[ L\left(y_i, \hat y_i^{(t-1)}\right) + g_if_t(x_i) + {1\over2}h_if_t^2(x_i) \right] + \Omega(f_t) + C \\
        &= \sum_{i=0}^n \left[g_if_t(x_i) + {1\over2}h_if_t^2(x_i) \right] + \Omega(f_t) + C \\
        &= \sum_{i=0}^n \left[g_iw_{q(x_i)} + {1\over2}h_i w_{q(x_i)}^2 \right] + \gamma T + {1\over 2}\lambda \sum_{j=1}^T w_j^2 + C \\
        &= \sum_{j=1}^T \left[\left(\sum_{i\in I_j}g_i\right) w_j + {1\over2}\left(\sum_{i\in I_j}h_i \right) w_j^2 \right] +  \gamma T + {1\over 2}\lambda \sum_{j=1}^T w_j^2 + C \\
         &= \sum_{j=1}^T \left[\left(\sum_{i\in I_j}g_i\right) w_j + {1\over2}\left(\sum_{i\in I_j}h_i + \lambda \right) w_j^2 \right] +  \gamma T + C \\
  \end{align}$$

* 定义：
   $G_j =\sum_{i\in I_j}g_i $, $H_j =\sum_{i\in I_j}h_i $
* 则目标函数可以写作：
    $$ J(f_t) =  \sum_{j=1}^T \left[G_j w_j + {1\over2} \left(H_j + \lambda\right) w_j^2 \right] +  \gamma T + C $$
* 对$w_j$ 求导,可以得到：
   $$\frac{\partial J(f_t)}{\partial w_j} = G_j + (H_j + \lambda)w_j = 0  \Rightarrow w_j =- \frac{G_j}{H_j+ \lambda}$$
   * 带入目标函数得到： $$ J(f_t) = - {1\over2} \sum_{i=1}^T \frac{G_j^2}{H_j + \lambda} + \gamma T $$

##### 子树划分
* 枚举可行的分割点，选择增益最大的划分，继续同样的操作，知道达到某个阈值或者为纯节点：
$$Gain(\phi) = {1\over2} \left[ \frac{G_L^2}{H_L + \lambda} + \frac{G_R^2}{H_R + \lambda} - \frac{(G_L + G_R)^2}{(H_L + H_R) + \lambda}\right]  - \gamma$$

#### GBDT 优缺点
- GBDT主要的优点有：
 - 可以灵活处理各种类型的数据，包括连续值和离散值。
 - 在相对少的调参时间情况下，预测的准备率也可以比较高。这个是相对SVM来说的。
 - 使用一些健壮的损失函数，对异常值的鲁棒性非常强。比如 Huber损失函数和Quantile损失函数。
- GBDT的主要缺点有：
 - 由于弱学习器之间存在依赖关系，难以并行训练数据。不过可以通过自采样的SGBT来达到部分并行。

### Adaboost

adaboost
 